### different disasters = different datasets
### each dataset = by country and year
### country, year, total people affected, total deaths, total damages in usd


In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('EMDAT.csv', skipinitialspace=True, skiprows=6)

/tmp/ipykernel_10080/1074600298.py:1: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('EMDAT.csv', skipinitialspace=True, skiprows=6)


In [3]:
disasters = pd.unique(df1['Disaster Type'])
countries = pd.unique(df1['Country'])
print('# Disaster types: ', len(disasters) )
print('# Countries : ', len(countries) )
#NaNs in interested columns tobe considered 0
df1['Total Deaths'].fillna(0, inplace = True)
df1['Total Affected'].fillna(0, inplace = True)
df1["Total Damages, Adjusted ('000 US$)"].fillna(0, inplace = True)
start_year = 1985
end_year = 2020


# Disaster types:  18
# Countries :  231


In [5]:
for disaster in disasters:    
    print("Processing disaster: ", disaster)
    disaster_all_dataframes = []
    for country in countries:
        sub_index = (df1['Country'] == country) & (df1['Disaster Type'] == disaster)
        single_year_events = sub_index & (df1['End Year'] == df1['Start Year'])
        multi_year_events = np.where(sub_index & (df1['End Year'] != df1['Start Year']))[0]
        list_of_points = []
        for year in range(start_year, end_year + 1):
            indices = single_year_events & (df1['Year'] == year)
            total_deaths = sum(df1['Total Deaths'][indices])
            total_affected = sum(df1['Total Affected'][indices])
            total_damages_adj = sum(df1["Total Damages, Adjusted ('000 US$)"][indices])
            list_of_points.append({'country':country, 'year':year, 'total_deaths':total_deaths, 'total_affected':total_affected, 'total_damages_adj':total_damages_adj})
        disaster_df = pd.DataFrame.from_records(list_of_points)
        for event in multi_year_events:
            n_years = df1.iloc[event]['End Year'] - df1.iloc[event]['Start Year'] + 1.0
            damage = df1.iloc[event]["Total Damages, Adjusted ('000 US$)"].copy() / n_years
            affected = df1.iloc[event]["Total Affected"] / n_years
            deaths = df1.iloc[event]["Total Deaths"] / n_years
            for year in range( df1.iloc[event]['Start Year'],  df1.iloc[event]['End Year'] + 1):
                if np.where(disaster_df['year'] == year)[0].size>0:
                    r = np.where(disaster_df['year'] == year)[0][0]
                    disaster_df.at[r,'total_deaths'] += deaths
                    disaster_df.at[r,'total_affected'] +=  affected
                    disaster_df.at[r,'total_damages_adj'] += damage
        disaster_all_dataframes.append(disaster_df)
    disaster_all_countries = pd.concat(disaster_all_dataframes)
#     disaster_all_countries.to_csv('/Volumes/NJ_4TB/PATSAT/MyFiles/' + disaster + '_EMDAT_panel.csv', encoding='utf-8', index=False) 
    disaster_all_countries.to_csv(disaster + '_EMDAT_panel.csv', encoding='utf-8', index=False) 

Processing disaster:  Storm
Processing disaster:  Earthquake
Processing disaster:  Landslide
Processing disaster:  Flood
Processing disaster:  Wildfire
Processing disaster:  Epidemic
Processing disaster:  Transport accident
Processing disaster:  Industrial accident
Processing disaster:  Miscellaneous accident
Processing disaster:  Drought
Processing disaster:  Volcanic activity
Processing disaster:  Extreme temperature 
Processing disaster:  Insect infestation
Processing disaster:  Mass movement (dry)
Processing disaster:  Complex Disasters
Processing disaster:  Impact
Processing disaster:  Animal accident
Processing disaster:  Glacial lake outburst
